In [108]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# from scipy.stats import sem
import os

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# File to Load
taxi_201801_csv = os.path.join("TaxiData", "yellow_tripdata_201801.csv")
taxi_201802_csv = os.path.join("TaxiData", "yellow_tripdata_2018-02.csv")
taxi_201803_csv = os.path.join("TaxiData", "yellow_tripdata_2018-03.csv")
taxi_201804_csv = os.path.join("TaxiData", "yellow_tripdata_2018-04.csv")
taxi_201805_csv = os.path.join("TaxiData", "yellow_tripdata_2018-05.csv")
taxi_201806_csv = os.path.join("TaxiData", "yellow_tripdata_2018-06.csv")
taxi_201807_csv = os.path.join("TaxiData", "yellow_tripdata_2018-07.csv")
taxi_201808_csv = os.path.join("TaxiData", "yellow_tripdata_2018-08.csv")
taxi_201809_csv = os.path.join("TaxiData", "yellow_tripdata_2018-09.csv")
taxi_201810_csv = os.path.join("TaxiData", "yellow_tripdata_2018-10.csv")
taxi_201811_csv = os.path.join("TaxiData", "yellow_tripdata_2018-11.csv")
taxi_201812_csv = os.path.join("TaxiData", "yellow_tripdata_2018-12.csv")

taxi_zone_lkup_csv = os.path.join("TaxiData", "taxi_zone_lookup.csv")

# Read the Taxi Data
taxi_201801_df = pd.read_csv(taxi_201801_csv)
taxi_zone_lkup_df = pd.read_csv(taxi_zone_lkup_csv)

taxi_zone_lkup_df.head()


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [105]:
# taxi_zone_lkup_df.nunique()
taxi_zone_dic = pd.Series(taxi_zone_lkup_df.Zone.values, index=taxi_zone_lkup_df.LocationID).to_dict()
taxi_zone_dic
taxi_201801_df['PU_zone_name'] = taxi_201801_df.PULocationID.map(taxi_zone_dic)
taxi_201801_df['DO_zone_name'] = taxi_201801_df.DOLocationID.map(taxi_zone_dic)

taxi_201801_df['trip_time'] = pd.to_datetime(taxi_201801_df["tpep_dropoff_datetime"]) - pd.to_datetime(taxi_201801_df["tpep_pickup_datetime"])

taxi_201801_df['trip_time_min'] = taxi_201801_df['trip_time'].dt.total_seconds()/60 

taxi_201801_df.head()



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PU_zone_name,DO_zone_name,trip_time,trip_time_min
0,1,2018-03-01 00:01:34,2018-03-01 00:01:43,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,Long Island City/Hunters Point,Long Island City/Hunters Point,00:00:09,0.150000
1,1,2018-03-01 00:14:34,2018-03-01 00:28:13,1,3.3,1,N,151,244,1,...,0.5,0.5,2.85,0.0,0.3,17.15,Manhattan Valley,Washington Heights South,00:13:39,13.650000
2,1,2018-03-01 00:51:25,2018-03-01 00:59:54,1,2.7,1,N,238,152,2,...,0.5,0.5,0.00,0.0,0.3,11.30,Upper West Side North,Manhattanville,00:08:29,8.483333
3,1,2018-03-01 00:00:01,2018-03-01 00:00:17,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,Long Island City/Hunters Point,Long Island City/Hunters Point,00:00:16,0.266667
4,1,2018-03-01 00:55:10,2018-03-01 00:56:36,1,3.7,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,4.30,Long Island City/Hunters Point,Long Island City/Hunters Point,00:01:26,1.433333


In [107]:

groupby_PUzone_taxi_201801 = taxi_201801_df.groupby(["PU_zone_name"])
gb_PU_trip_count = groupby_PUzone_taxi_201801['PULocationID'].count()
gb_PU_trip_rev_avg = groupby_PUzone_taxi_201801['total_amount'].mean()
gb_PU_trip_rev_total = groupby_PUzone_taxi_201801['total_amount'].sum()
gb_PU_trip_time_total = groupby_PUzone_taxi_201801['trip_time_min'].sum()
gb_PU_trip_time_avg = groupby_PUzone_taxi_201801['trip_time_min'].mean()
gb_PU_trip_miles_total = groupby_PUzone_taxi_201801['trip_distance'].sum()
gb_PU_trip_miles_avg = groupby_PUzone_taxi_201801['trip_distance'].mean()

gb_PU_summary_df = pd.DataFrame({"trip_count": gb_PU_trip_count,
                             "avg_revenue": gb_PU_trip_rev_avg,
                             "sum_revenue": gb_PU_trip_rev_total,
                             "total_time_min": gb_PU_trip_time_total,
                             "avg_time_min": gb_PU_trip_time_avg,
                             "total_miles": gb_PU_trip_miles_total,
                             "avg_miles": gb_PU_trip_miles_avg})

gb_PU_summary_df['avg_rev_per_min'] = gb_PU_summary_df['sum_revenue']/gb_PU_summary_df['total_time_min'] 
gb_PU_summary_df['avg_rev_per_min']


gb_PU_summary_df = gb_PU_summary_df.sort_values('avg_rev_per_min', ascending=False)

jfk_df = gb_PU_summary_df.loc[['JFK Airport','Newark Airport','LaGuardia Airport']]

jfk_df

# gb_PU_summary_df

,trip_count,avg_revenue,sum_revenue,total_time_min,avg_time_min,total_miles,avg_miles,avg_rev_per_min
PU_zone_name,,,,,,,,
Broad Channel,2,35.300000,7.060000e+01,4.166667e-01,0.208333,92.00,46.000000,169.440000
Charleston/Tottenville,2,64.950000,1.299000e+02,9.166667e-01,0.458333,29.90,14.950000,141.709091
Westerleigh,3,55.270000,1.658100e+02,1.366667e+00,0.455556,0.00,0.000000,121.324390
Great Kills,2,107.855000,2.157100e+02,5.583333e+00,2.791667,0.00,0.000000,38.634627
Newark Airport,659,89.208331,5.878829e+04,2.212100e+03,3.356753,1435.96,2.178998,26.575783
Eltingville/Annadale/Prince's Bay,7,55.190000,3.863300e+02,2.293333e+01,3.276190,0.69,0.098571,16.845785
Grymes Hill/Clifton,5,35.304000,1.765200e+02,1.665000e+01,3.330000,1.59,0.318000,10.601802
Rossville/Woodrow,3,70.966667,2.129000e+02,2.048333e+01,6.827778,6.60,2.200000,10.393816
Arden Heights,4,34.420000,1.376800e+02,2.561667e+01,6.404167,12.84,3.210000,5.374626
